In [14]:
import numpy as np

from utils import *

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import KFold,GridSearchCV

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from joblib import Parallel, delayed

In [2]:
# Rutas de los datos
ruta_entrenamiento = "db/KDDTrain+.arff"
ruta_prueba = "db/KDDTest+.arff"
ruta_entrenamiento, ruta_prueba

('db/KDDTrain+.arff', 'db/KDDTest+.arff')

In [3]:
X_entrenamiento, y_entrenamiento = procesar_datos_entrada(ruta_entrenamiento)
X_prueba, y_prueba = procesar_datos_entrada(ruta_prueba)

print("Dimensiones de las características de entrenamiento:",
    X_entrenamiento.shape)
print("Dimensiones de las características de prueba:", X_prueba.shape)

Dimensiones de las características de entrenamiento: (125973, 41)
Dimensiones de las características de prueba: (22544, 41)


In [4]:
preprocesador = obtener_preprocesador(X_entrenamiento)
preprocesador

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 Index(['count', 'diff_srv_rate', 'dst_bytes', 'dst_host_count',
       'dst_host_diff_srv_rate', 'dst_host_rerror_rate',
       'dst_host_same_src_port_rate', 'dst_host_same_srv_rate',
       'dst_host_serror_rate', 'dst_host_srv_count',
       'dst_host_srv_diff_host_rate', 'dst_host_srv_rerror_rate',
       'dst_host_srv_serror_rate', 'du...
       'num_outbound_cmds', 'num_root', 'num_shells', 'rerror_rate',
       'root_shell', 'same_srv_rate', 'serror_rate', 'src_bytes', 'srv_count',
       'srv_diff_host_rate', 'srv_rerror_rate', 'srv_serror_rate',
       'su_attempted', 'urgent', 'wrong_fragment'],
      dtype='object')),
                                ('cat', OneHotEncoder(handle_unknown='ignore'),
                                 Index(['protocol_type', 'service', 'flag', 'land', 'logged_in',
       'is_host_login', 'is_guest_login'],
      dtype='object'))])

In [5]:
def regresion_polinomica(modelo):
    return Pipeline([
        ('polinomio', PolynomialFeatures()),
        ('modelo', modelo)
    ])

modelos = {
    'Regresión lineal': {
        'modelo': LinearRegression(),
        'params': {}
    },
    'Ridge': {
        'modelo': Ridge(),
        'params': {
            'regresor__alpha': [0.1, 1.0, 10.0, 100.0]
        }
    },
    'Lasso': {
        'modelo': Lasso(),
        'params': {
            'regresor__alpha': [0.1, 1.0, 10.0, 100.0]
        }
    },
    'Regresión polinómica': {
        'modelo': regresion_polinomica(LinearRegression()),
        'params': {
            'regresor__polinomio__degree': [2, 3]
        }
    },
    'Regresión polinómica (ridge)': {
        'modelo': regresion_polinomica(Ridge()),
        'params': {
            'regresor__polinomio__degree': [2, 3],
            'regresor__modelo__alpha': [0.1, 1.0]
        }
    },
    'Regresión polinómica (lasso)': {
        'modelo': regresion_polinomica(Lasso()),
        'params': {
            'regresor__polinomio__degree': [2, 3],
            'regresor__modelo__alpha': [0.1, 1.0]
        }
    },
    'KNN': {
        'modelo': KNeighborsRegressor(),
        'params': {
            'regresor__n_neighbors': [3, 5, 7, 9],
            'regresor__weights': ['uniform', 'distance']
        }
    },
    'Árbol de decisión': {
        'modelo': DecisionTreeRegressor(),
        'params': {
            'regresor__max_depth': [3, 5, 7, None],
            'regresor__min_samples_split': [2, 5, 10],
            'regresor__min_samples_leaf': [1, 2, 4]
        }
    },
    'Bosque aleatorio': {
        'modelo': RandomForestRegressor(),
        'params': {
            'regresor__n_estimators': [100, 200, 300],
            'regresor__max_features': ['sqrt', 'log2', None],
            'regresor__max_depth': [None, 10, 20, 30],
            'regresor__min_samples_split': [2, 5, 10],
            'regresor__min_samples_leaf': [1, 2, 4]
        }
    }
}

In [6]:
# Configurar la validación cruzada
kf = KFold(n_splits=3, shuffle=True, random_state=42)
X_muestra = X_entrenamiento.sample(n=400, random_state=42)
y_muestra = y_entrenamiento.sample(n=400, random_state=42)

def evaluar_modelo(nombre, modelo):
    pipeline = Pipeline(steps=[
        ('preprocesador', preprocesador),
        ('regresor', modelo['modelo'])
    ])

    # Configurar búsqueda de hiperparámetros
    grid_search = GridSearchCV(pipeline, modelo['params'], cv=kf, scoring='r2',
        n_jobs=-1)

    # Realizar la búsqueda de hiperparámetros
    grid_search.fit(X_muestra, y_muestra)

    # Imprimir los resultados
    print(f"{nombre}: Mejor R2 = {grid_search.best_score_:.4f}")
    print(f"{nombre}: Mejores parámetros = {grid_search.best_params_}")
    print()
    
    # Guardar los mejores resultados
    return nombre, {
        'mejor_r2': grid_search.best_score_,
        'mejores_params': grid_search.best_params_
    }

In [7]:
# Ejecutar la evaluación de los modelos en paralelo
resultados = Parallel(n_jobs=-1)(
    delayed(evaluar_modelo)(nombre, modelo)
        for nombre, modelo in modelos.items()
)

Regresión lineal: Mejor R2 = 0.0460
Regresión lineal: Mejores parámetros = {}

Lasso: Mejor R2 = 0.6057
Lasso: Mejores parámetros = {'regresor__alpha': 0.1}

Ridge: Mejor R2 = 0.3955
Ridge: Mejores parámetros = {'regresor__alpha': 1.0}

KNN: Mejor R2 = 0.8807
KNN: Mejores parámetros = {'regresor__n_neighbors': 7, 'regresor__weights': 'distance'}

Árbol de decisión: Mejor R2 = 0.8263
Árbol de decisión: Mejores parámetros = {'regresor__max_depth': 7, 'regresor__min_samples_leaf': 2, 'regresor__min_samples_split': 5}

Regresión polinómica (ridge): Mejor R2 = -191390.0689
Regresión polinómica (ridge): Mejores parámetros = {'regresor__modelo__alpha': 0.1, 'regresor__polinomio__degree': 2}



/usr/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.512e-02, tolerance: 6.596e-03
  model = cd_fast.enet_coordinate_descent(
/usr/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.158e-02, tolerance: 6.560e-03
  model = cd_fast.enet_coordinate_descent(


Regresión polinómica (lasso): Mejor R2 = -0.0012
Regresión polinómica (lasso): Mejores parámetros = {'regresor__modelo__alpha': 1.0, 'regresor__polinomio__degree': 2}

Regresión polinómica: Mejor R2 = -389125.9961
Regresión polinómica: Mejores parámetros = {'regresor__polinomio__degree': 2}

Bosque aleatorio: Mejor R2 = 0.9255
Bosque aleatorio: Mejores parámetros = {'regresor__max_depth': 20, 'regresor__max_features': 'sqrt', 'regresor__min_samples_leaf': 1, 'regresor__min_samples_split': 2, 'regresor__n_estimators': 200}



In [8]:
# Configurar la tubería con los mejores parámetros
pipeline = Pipeline(steps=[
    ('preprocesador', preprocesador),
    ('regresor', RandomForestRegressor(
        max_depth=20,
        max_features='sqrt',
        min_samples_leaf=1,
        min_samples_split=2,
        n_estimators=200
    ))
])

In [9]:
# Ajustar el modelo a los datos de entrenamiento
pipeline.fit(X_entrenamiento, y_entrenamiento)

Pipeline(steps=[('preprocesador',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['count', 'diff_srv_rate', 'dst_bytes', 'dst_host_count',
       'dst_host_diff_srv_rate', 'dst_host_rerror_rate',
       'dst_host_same_src_port_rate', 'dst_host_same_srv_rate',
       'dst_host_serror_rate', 'dst_host_srv_count',
       'dst_host_srv_diff_host_rate', 'dst_host_srv_rerror_rat...
       'srv_diff_host_rate', 'srv_rerror_rate', 'srv_serror_rate',
       'su_attempted', 'urgent', 'wrong_fragment'],
      dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['protocol_type', 'service', 'flag', 'land', 'logged_in',
       'is_host_login', 'is_guest_login'],
      dtype='object'))])),
                ('regresor',
                 RandomForestRegressor(max_depth=20, max_features='sqrt',
                                       n_estimators=200))])

In [10]:
# Evaluar el modelo con los datos de prueba
y_pred = pipeline.predict(X_prueba)
y_pred[:5]

array([1.        , 1.        , 0.0012573 , 0.94748216, 0.35875196])

In [16]:
# Evaluar el R2 en los datos de prueba
r2 = r2_score(y_prueba, y_pred)
ecm = mean_squared_error(y_prueba, y_pred)
recm = np.sqrt(ecm)
eam = mean_absolute_error(y_prueba, y_pred)

print(f"R² en los datos de prueba: {r2:.4f}")
print(f"Error cuadrático medio en los datos de prueba: {ecm:.4f}")
print(f"Raíz del error cuadrático medio en los datos de prueba: {recm:.4f}")
print(f"Error medio absoluto en los datos de prueba: {eam:.4f}")

R² en los datos de prueba: 0.3150
Error cuadrático medio en los datos de prueba: 0.1680
Raíz del error cuadrático medio en los datos de prueba: 0.4098
Error medio absoluto en los datos de prueba: 0.2152
